This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests


In [ ]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

http = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=%s" % API_KEY
# Ensure http request includes API_KEY correctly


print(http)
r =  requests.get(http)


data = r.json()['dataset_data']['data']
print(data[0])


In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())
print('----------------------------------')
print(data[:4])


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
# Variables used to set up Dictionary of 2017
r_2017 = []
dict_2017 = {}
data_columns = r.json()['dataset_data']['column_names']


# Create list of 2017 values
for value in data:
    if value[0].startswith('2017'):
        r_2017.append(value)

        
# Using date as key, set up nested dictionary with column names as keys and respective values as the values
for value in r_2017:
    delete_value = data_columns[0]
    temp_dict=dict(zip(data_columns, value))
    del temp_dict[delete_value]
    dict_2017[value[0]] = temp_dict
 

print(dict_2017)       


In [ ]:
# Set up initial variables for analysis
high_close_value = 0
high_close_date = ''

low_close_value = 100
low_close_date = ''

one_day_difference = 0
one_day_difference_date = ''

previous_close = dict_2017['2017-12-29']['Close']
close_difference = 0
close_difference_dates = ''

avg_count = 0
avg_volume_count = 0
avg_volume = 0


# Loop through the items inside of dictionary
for key, value in dict_2017.items():
    
    # Find highest closing value
    if value['Close'] > high_close_value:
        high_close_date = key
        high_close_value = value['Close']
     
    # Find lowest closing value
    if value['Close'] < low_close_value:
        low_close_date = key
        low_close_value = value['Close']
    
    # Ensure both 'Close' and 'Open' have numeric values
    if(type(value['Close']) == float and type(value['Open']) == float): 
        
        # Find biggest difference from open to close in one day
        computed_one_day_difference = abs(value['Close'] - value['Open'])
        if computed_one_day_difference > one_day_difference:
            one_day_difference = computed_one_day_difference
            one_day_difference_date = key
    
    # Find biggest difference between consecutive days
    computed_difference = abs(value['Close']-previous_close)
    if (computed_difference > close_difference):
        close_difference = computed_difference
        close_difference_dates = [previous_close_date, key]
    
    # Set closing value and date for analysis of consecutive days on next iteration of for loop
    previous_close = value['Close']
    previous_close_date = key
    
    # Find average volume traded over the 2017 year
    avg_count+=1
    avg_volume_count += value['Traded Volume']
    avg_volume= avg_volume_count/avg_count
    
  
# print statements to show values found in for loop above
print('The highest closing value of 2017 was', high_close_value, 'on', high_close_date)
print('The lowest closing value of 2017 was', low_close_value, 'on', low_close_date)
print('The biggest one day difference between open and close was', round(one_day_difference,2), 'on', one_day_difference_date)
print('The biggest difference between two closes was', round(close_difference, 2), 'between the days of', close_difference_dates[0], 'and', close_difference_dates[1])
print('The average volume per day traded in 2017 was', round(avg_volume, 2))
